# Map examples

This notebook loads some simple GeoJSON data containing lines for submarine cables from [submarinecablemap.com](http://www.submarinecablemap.com) and displays them on various maps, where you can pan and zoom and, sometimes, also tilt and rotate the map.

In [ ]:
import requests

url = (
    "https://raw.githubusercontent.com/telegeography/www.submarinecablemap.com"
    "/master/web/public/api/v3/cable/cable-geo.json"
)
data = requests.get(url).json()

## Ipyleaflet

In [ ]:
from ipyleaflet import Map, GeoJSON

m = Map(center=[0, 0], zoom=2)
m += GeoJSON(
    data=data,
    style_callback=lambda feat: {"color": feat["properties"]["color"]},
    name = 'Cables'
)
m

## Folium

In [ ]:
import folium

m = folium.Map(
    location=[0, 0],
    zoom_start=2,
)
geojson = folium.GeoJson(
    data=data,
    style_function=lambda feat: {"color": feat["properties"]["color"]}
)
geojson.add_to(m)
m

## Geemap

This needs a Google Earth Engine registration.

In [ ]:
import geemap

m = geemap.Map(center=[0, 0], zoom=2, height="450px")
m.add_geojson(
    data,
    style_callback=lambda feat: {"color": feat["properties"]["color"]}
)
m

## Leafmap

The `style_callback` parameter works as expected in leafmap>0.5.0.

In [ ]:
import leafmap

m = leafmap.Map(center=[0, 0], zoom=2, height="450px")
m.add_geojson(
    data,
    style_callback=lambda feat: {"color": feat["properties"]["color"]}
)
m

## HERE Map Widget

This reads an API key from the ``HEREMAPS_API_KEY`` environment variable. Set it below or provide the API key directly in the code, instead:

In [ ]:
import os
os.environ["HEREMAPS_API_KEY"] = "MY-API-TOKEN"

In [ ]:
import os

from here_map_widget import Map, GeoJSON

api_key = os.getenv("HEREMAPS_API_KEY")
m = Map(center=[0, 0], zoom=1, api_key=api_key)
m += GeoJSON(
    data=data,
    style_function=lambda feat: {"strokeColor": feat["properties"]["color"]},
    name = 'Cables'
)
m

## HERE Inspector (ipyleaflet)

In [ ]:
from here.inspector import inspect

inspect(data, zoom=2) # , style=lambda feat: {"color": feat["properties"]["color"]})

## Deck.gl

In [ ]:
import pydeck

pydeck.settings.custom_libraries = [
    {
        "libraryName": "LabeledGeoJsonLayerLibrary",
        "resourceUri": "https://unpkg.com/pydeck-custom-layer-demo/dist/bundle.js",
    }
]

custom_layer = pydeck.Layer(
    "LabeledGeoJsonLayer",
    data=url,
    filled=False,
    billboard=False,
    get_line_color=[180, 180, 180],
    get_label="properties.name",
    get_label_size=200000,
    get_label_color=[0, 255, 255],
    label_size_units=pydeck.types.String("meters"),
    line_width_min_pixels=1,
)

view_state = pydeck.ViewState(latitude=0, longitude=0, zoom=1)
r = pydeck.Deck(custom_layer, initial_view_state=view_state, map_provider=None)
r.to_html("cables_deckgl.html", css_background_color="#333")